In [ ]:
# Show simulation results
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
# from utils import Re

In [ ]:
def Re(theta):
    t = np.deg2rad(theta)
    c = np.cos(t)
    s = np.sin(t)
    R = [[c**2, s**2, c*s], [s**2, c**2, -c*s], [-2*c*s, 2*c*s, c**2 - s**2]]
    return R

In [ ]:
theta = 5
filename = 'Elastic_orthotropic_balanced\\tensile_{}.mat'.format(theta)
data = loadmat(filename)
n_frames = data['EPS'].shape[0]/data['nelem']

In [ ]:
eps_l = data['EPS']
eps_p = np.matmul(Re(theta), eps_l.T).T
# Plot strains in principal ref. system
colors = cm.viridis(np.linspace(0, 1, 45))
plt.scatter(eps_p[:, 1], eps_p[:, 2], label='$\\theta$ ={}'.format(theta), marker='.', s=10, color=colors[theta], alpha=.4)
plt.xlabel(r'$\varepsilon_{2}$')
plt.ylabel('$\gamma_{12}$')
plt.legend()
plt.show()

In [ ]:
## Apply damage to matrix
gamma0 = 0.04
gammaf = 0.4
gamma = eps_p[:, 2]
gamma = np.linspace(0, 0.2, 50)
G12 = 2.5
d12 = np.multiply((gamma>=gamma0),np.divide((gamma-gamma0),(gammaf-gamma0)))**0.7
tau = G12*gamma - np.multiply(d12*G12, gamma)
tau[gamma>gammaf] = 0
plt.figure()
plt.scatter(gamma, tau*10, marker='.')
plt.scatter(gamma, gamma*G12)
plt.scatter(gamma, d12)
print('{} {}'.format(gamma.T, tau.T))
np.savetxt("array.txt", np.vstack((gamma, tau)).T, fmt='%20f', delimiter='')

In [ ]:
# Plot strains at frame N
N = 80
nelem = data['nelem'][0][0].astype(int)
plt.figure()
print()
plt.scatter(data['X'], data['Y'], c=eps_l[N*nelem:(N+1)*nelem, 1])
plt.axis('equal')
plt.colorbar()
plt.show()

In [ ]:
# Plot divergence
from matplotlib.tri import *
tri = Triangulation(data['X'].flatten(), data['Y'].flatten())
Fx = LinearTriInterpolator(tri, eps_l[N*nelem:(N+1)*nelem, 0]+eps_l[N*nelem:(N+1)*nelem, 2])
Fy = LinearTriInterpolator(tri, eps_l[N*nelem:(N+1)*nelem, 1]+eps_l[N*nelem:(N+1)*nelem, 2])
gradx = Fx.gradient(data['X'], data['Y'])
grady = Fy.gradient(data['X'], data['Y'])
plt.scatter(data['X'], data['Y'], c=gradx[0]+grady[0]+gradx[1]+grady[1])
plt.axis('equal')
plt.colorbar()
plt.show()

In [ ]:
gxy = np.linspace(0, 0.22, 5000)
noise = np.random.normal(0, 0.08*np.sqrt(gxy), len(gxy))
G12 = 2.22 + np.zeros(gxy.shape)
G12[gxy>=0.05] = 2.22 - np.multiply(2.5/0.15, gxy[gxy>=0.05]-0.05)
G12 = G12 + noise
G12[gxy>=0.2] = 0
txy_e = np.multiply(gxy, G12)
# txy_e = 2.5*gxy
# txy_e[gxy>=0.05] = txy_e[gxy>=0.05] - gxy[gxy>=0.05]*np.multiply(2.5/0.15, gxy[gxy>=0.05]-0.05)
# txy_e[gxy>=0.2] = 0

import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Helvetica"
plt.figure(dpi=150, figsize = [3, 3])
plt.scatter(gxy, (txy_e), marker='.', s=2, alpha=.3, label='$\gamma_{12}$ - $\\tau_{12}$')
plt.xlim(0, 0.25), plt.ylim(0)
plt.xlabel('Strain'), plt.ylabel('Stress [GPa]')
plt.legend(loc='best')
plt.tight_layout()
ax = plt.gca()
from matplotlib.ticker import FuncFormatter

def format_fn(tick_val, tick_pos):
    return "{:.2f}".format(tick_val)

ax.yaxis.set_major_formatter(FuncFormatter(format_fn))
plt.savefig('Std_test_dmg.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Helvetica"
plt.figure(dpi=150, figsize = [3, 3])
plt.scatter(gxy, np.divide(txy_e, gxy), marker='.', s=2, alpha=.3, label='$\gamma_{12}$ - $G_{12}$')
plt.xlim(0, 0.25), plt.ylim(0)
plt.xlabel('Strain'), plt.ylabel('Stiffness [GPa]')
plt.legend(loc='best')
plt.tight_layout()
ax = plt.gca()
from matplotlib.ticker import FuncFormatter

def format_fn(tick_val, tick_pos):
    return "{:.2f}".format(tick_val)

ax.yaxis.set_major_formatter(FuncFormatter(format_fn))
plt.savefig('Std_test_dmg_G12.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        metadata=None)